<a href="https://colab.research.google.com/github/Julio-0204/Python-Lector-de-Placas-Vehiculares/blob/main/Evaluacion_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract openpyxl

In [ ]:
# Instalar dependencias
import os
import sys

os.system('apt-get install -y tesseract-ocr > /dev/null 2>&1')
os.system('pip install pytesseract openpyxl > /dev/null 2>&1')

# Importar librerías
import cv2
import numpy as np
import pandas as pd
import pytesseract
from datetime import datetime, timedelta
from google.colab.patches import cv2_imshow
from google.colab import files
from IPython.display import display, Javascript, HTML
from base64 import b64decode
import io
from PIL import Image as PILImage
import re
import time

# Configurar zona horaria de Colombia (UTC-5)
ZONA_HORARIA_COLOMBIA = timedelta(hours=-5)

# ============================================
# CONFIGURACIÓN
# ============================================

ARCHIVO_EXCEL = "placas_registradas.xlsx"
CARPETA_IMAGENES = "placas_capturadas"
CONFIGURACION_TESSERACT = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

# Variables globales
sistema_activo = True
contador_sesion = 0

# ============================================
# FUNCIONES DE BASE DE DATOS
# ============================================

def inicializar_sistema():
    """Crea la estructura inicial del sistema."""
    if not os.path.exists(CARPETA_IMAGENES):
        os.makedirs(CARPETA_IMAGENES)

    if os.path.exists(ARCHIVO_EXCEL):
        return pd.read_excel(ARCHIVO_EXCEL)
    else:
        df = pd.DataFrame(columns=['Fecha', 'Hora', 'Placa', 'Confianza', 'Imagen'])
        df.to_excel(ARCHIVO_EXCEL, index=False)
        return df

def guardar_placa(placa, confianza, nombre_imagen):
    """Guarda el registro de la placa en el Excel con hora de Colombia."""
    df = pd.read_excel(ARCHIVO_EXCEL)

    # Obtener hora de Colombia (UTC-5)
    ahora_utc = datetime.utcnow()
    ahora_colombia = ahora_utc + ZONA_HORARIA_COLOMBIA

    nueva_fila = {
        'Fecha': ahora_colombia.strftime('%Y-%m-%d'),
        'Hora': ahora_colombia.strftime('%H:%M:%S'),
        'Placa': placa,
        'Confianza': f"{confianza}%",
        'Imagen': nombre_imagen
    }

    df = pd.concat([df, pd.DataFrame([nueva_fila])], ignore_index=True)
    df.to_excel(ARCHIVO_EXCEL, index=False)
    return df

# ============================================
# PROCESAMIENTO DE IMAGEN
# ============================================

def preprocesar_imagen_rapido(imagen):
    """Preprocesamiento optimizado."""
    height, width = imagen.shape[:2]
    if width > 1280:
        scale = 1280 / width
        imagen = cv2.resize(imagen, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    gris = cv2.bilateralFilter(gris, 9, 75, 75)
    bordes = cv2.Canny(gris, 50, 150)

    return imagen, gris, bordes

def encontrar_placa_mejorado(imagen, bordes):
    """Detección de placa mejorada."""
    contornos, _ = cv2.findContours(bordes.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contornos = sorted(contornos, key=cv2.contourArea, reverse=True)[:10]

    candidatos = []

    for contorno in contornos:
        perimetro = cv2.arcLength(contorno, True)
        aproximacion = cv2.approxPolyDP(contorno, 0.02 * perimetro, True)

        if len(aproximacion) == 4:
            x, y, w, h = cv2.boundingRect(aproximacion)
            aspect_ratio = w / float(h)
            area = w * h

            if (2.0 <= aspect_ratio <= 5.0 and w > 80 and h > 20 and area > 2000):
                candidatos.append((aproximacion, area))

    if candidatos:
        candidatos.sort(key=lambda x: x[1], reverse=True)
        return candidatos[0][0]

    return None

def mejorar_region_placa(placa_recortada):
    """Mejora la imagen de la placa."""
    placa = cv2.resize(placa_recortada, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)

    if len(placa.shape) == 3:
        placa = cv2.cvtColor(placa, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    placa = clahe.apply(placa)
    placa = cv2.fastNlMeansDenoising(placa, h=10)
    _, placa = cv2.threshold(placa, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    kernel = np.ones((2, 2), np.uint8)
    placa = cv2.morphologyEx(placa, cv2.MORPH_CLOSE, kernel)

    return placa

def extraer_texto_mejorado(imagen, ubicacion_placa):
    """Extracción de texto optimizada."""
    if ubicacion_placa is None:
        return None, None, 0

    mascara = np.zeros(imagen.shape[:2], np.uint8)
    cv2.drawContours(mascara, [ubicacion_placa], 0, 255, -1)

    (y, x) = np.where(mascara == 255)
    (y1, x1) = (np.min(y), np.min(x))
    (y2, x2) = (np.max(y), np.max(x))
    placa_recortada = imagen[y1:y2+1, x1:x2+1]

    placa_mejorada = mejorar_region_placa(placa_recortada)

    resultados = []

    config1 = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    texto1 = pytesseract.image_to_string(placa_mejorada, config=config1)
    texto_limpio1 = limpiar_texto(texto1)
    if texto_limpio1:
        resultados.append(texto_limpio1)

    config2 = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    texto2 = pytesseract.image_to_string(placa_mejorada, config=config2)
    texto_limpio2 = limpiar_texto(texto2)
    if texto_limpio2:
        resultados.append(texto_limpio2)

    placa_invertida = cv2.bitwise_not(placa_mejorada)
    texto3 = pytesseract.image_to_string(placa_invertida, config=config1)
    texto_limpio3 = limpiar_texto(texto3)
    if texto_limpio3:
        resultados.append(texto_limpio3)

    if resultados:
        mejor_resultado = max(set(resultados), key=resultados.count)
        confianza = (resultados.count(mejor_resultado) / len(resultados)) * 100
        return mejor_resultado, placa_mejorada, int(confianza)

    return None, placa_mejorada, 0

def limpiar_texto(texto):
    """Limpia y valida el texto con correcciones inteligentes."""
    if not texto:
        return None

    texto = re.sub(r'[^A-Z0-9]', '', texto.upper())

    # Correcciones solo para números confundidos con letras
    # NO corregimos letras que deberían ser letras (como S)
    correcciones_numeros = {
        'O': '0',  # Solo O -> 0
        'I': '1',  # Solo I -> 1
    }

    # Aplicar correcciones solo en la parte numérica (últimos 3-4 caracteres)
    # Formato típico: ABC123 o ABC1234
    if len(texto) >= 6:
        # Las últimas 3-4 posiciones suelen ser números
        parte_letras = texto[:-3]  # Primera parte (letras)
        parte_numeros = texto[-3:]  # Última parte (números)

        # Solo corregir O e I en la parte numérica
        for viejo, nuevo in correcciones_numeros.items():
            parte_numeros = parte_numeros.replace(viejo, nuevo)

        texto = parte_letras + parte_numeros

    if 5 <= len(texto) <= 8:
        return texto

    return None

def procesar_imagen(imagen):
    """Procesa una imagen y extrae la placa."""
    print("\n🔍 Analizando imagen...")
    inicio = time.time()

    imagen, gris, bordes = preprocesar_imagen_rapido(imagen)
    ubicacion_placa = encontrar_placa_mejorado(imagen, bordes)

    if ubicacion_placa is not None:
        print("  ✓ Placa detectada\n")

        imagen_resultado = imagen.copy()
        cv2.drawContours(imagen_resultado, [ubicacion_placa], -1, (0, 255, 0), 3)

        print("📸 Imagen con placa detectada:")
        cv2_imshow(imagen_resultado)

        texto_placa, placa_mejorada, confianza = extraer_texto_mejorado(imagen, ubicacion_placa)

        if placa_mejorada is not None:
            print("\n🔎 Región de la placa procesada:")
            cv2_imshow(placa_mejorada)

        if texto_placa:
            print(f"\n✅ PLACA RECONOCIDA: {texto_placa}")
            print(f"🎯 Confianza: {confianza}%")
            print(f"⏱️  Tiempo: {time.time() - inicio:.2f}s")
            return texto_placa, imagen_resultado, confianza
        else:
            print("\n⚠️ No se pudo leer el texto")
            return None, imagen_resultado, 0
    else:
        print("\n❌ No se detectó ninguna placa")
        return None, imagen, 0

# ============================================
# CAPTURA CON CÁMARA
# ============================================

def capturar_con_camara():
    """Captura imagen con la cámara web."""
    print("\n🎥 Abriendo cámara web...")
    print("   Presiona el botón para capturar\n")

    js_code = """
    async function captureImage() {
      const div = document.createElement('div');
      const video = document.createElement('video');
      const canvas = document.createElement('canvas');
      const button = document.createElement('button');

      div.style.textAlign = 'center';
      button.textContent = '📸 CAPTURAR FOTO';
      button.style.fontSize = '20px';
      button.style.padding = '15px 30px';
      button.style.margin = '10px';
      button.style.backgroundColor = '#4CAF50';
      button.style.color = 'white';
      button.style.border = 'none';
      button.style.borderRadius = '5px';
      button.style.cursor = 'pointer';

      video.style.maxWidth = '100%';
      video.style.border = '3px solid #4CAF50';

      div.appendChild(video);
      div.appendChild(document.createElement('br'));
      div.appendChild(button);
      document.body.appendChild(div);

      const stream = await navigator.mediaDevices.getUserMedia({
        video: {facingMode: 'environment', width: {ideal: 1280}, height: {ideal: 720}}
      });
      video.srcObject = stream;
      await video.play();

      await new Promise(resolve => {
        button.onclick = () => resolve();
      });

      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getTracks().forEach(track => track.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg', 0.95);
    }
    """

    try:
        display(Javascript(js_code))
        from google.colab import output
        data = output.eval_js('captureImage()')

        binary = b64decode(data.split(',')[1])
        img = PILImage.open(io.BytesIO(binary))
        img_array = np.array(img)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        return img_array
    except Exception as e:
        print(f"❌ Error al capturar: {e}")
        return None

# ============================================
# SISTEMA PRINCIPAL CON BUCLE
# ============================================

def sistema_principal():
    """Sistema principal con menú interactivo en bucle."""
    global contador_sesion

    print("\n" + "=" * 70)
    print("   🚗 SISTEMA DE RECONOCIMIENTO DE PLACAS V3.0 🚗")
    print("=" * 70)

    inicializar_sistema()

    while True:
        print("\n" + "=" * 70)
        print("   MENÚ PRINCIPAL")
        print("=" * 70)
        print("\n  Selecciona una opción:\n")
        print("    1. 📸 Capturar placa con cámara")
        print("    2. 📤 Subir imagen de placa")
        print("    3. 📊 Ver registros")
        print("    4. ✏️  Corregir registro")
        print("    5. 💾 Descargar Excel")
        print("    6. 🖼️  Descargar imágenes")
        print("    7. ❌ Salir")
        print("\n" + "=" * 70)

        try:
            opcion = input("\n➤ Ingresa tu opción (1-7): ").strip()

            if opcion == '1':
                # CAPTURAR CON CÁMARA
                print("\n" + "-" * 70)
                print("   OPCIÓN 1: CAPTURAR CON CÁMARA")
                print("-" * 70)

                imagen = capturar_con_camara()

                if imagen is not None:
                    placa, imagen_procesada, confianza = procesar_imagen(imagen)

                    if placa:
                        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                        nombre_imagen = f"placa_{timestamp}.jpg"
                        ruta_imagen = os.path.join(CARPETA_IMAGENES, nombre_imagen)
                        cv2.imwrite(ruta_imagen, imagen_procesada)

                        guardar_placa(placa, confianza, nombre_imagen)
                        contador_sesion += 1

                        print(f"\n💾 ¡GUARDADO! Registro #{contador_sesion}")
                        print(f"   🔢 Placa: {placa}")
                        print(f"   🎯 Confianza: {confianza}%")
                    else:
                        print("\n⚠️ No se pudo registrar la placa")

                input("\n[Presiona ENTER para continuar...]")

            elif opcion == '2':
                # SUBIR IMAGEN
                print("\n" + "-" * 70)
                print("   OPCIÓN 2: SUBIR IMAGEN")
                print("-" * 70)
                print("\n📤 Sube una imagen de placa\n")

                uploaded = files.upload()

                if uploaded:
                    for nombre_archivo in uploaded.keys():
                        print(f"\n📂 Procesando: {nombre_archivo}")

                        imagen_bytes = uploaded[nombre_archivo]
                        imagen = cv2.imdecode(np.frombuffer(imagen_bytes, np.uint8), cv2.IMREAD_COLOR)

                        placa, imagen_procesada, confianza = procesar_imagen(imagen)

                        if placa:
                            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                            nombre_imagen = f"placa_{timestamp}.jpg"
                            ruta_imagen = os.path.join(CARPETA_IMAGENES, nombre_imagen)
                            cv2.imwrite(ruta_imagen, imagen_procesada)

                            guardar_placa(placa, confianza, nombre_imagen)
                            contador_sesion += 1

                            print(f"\n💾 ¡GUARDADO! Registro #{contador_sesion}")
                            print(f"   🔢 Placa: {placa}")
                            print(f"   🎯 Confianza: {confianza}%")
                        else:
                            print("\n⚠️ No se pudo registrar la placa")

                input("\n[Presiona ENTER para continuar...]")

            elif opcion == '3':
                # VER REGISTROS
                print("\n" + "-" * 70)
                print("   OPCIÓN 3: VER REGISTROS")
                print("-" * 70 + "\n")

                if os.path.exists(ARCHIVO_EXCEL):
                    df = pd.read_excel(ARCHIVO_EXCEL)
                    if len(df) > 0:
                        display(df)
                        print(f"\n📈 Total de registros: {len(df)}")

                        if 'Confianza' in df.columns:
                            df['Confianza_num'] = df['Confianza'].str.replace('%', '').astype(float)
                            promedio = df['Confianza_num'].mean()
                            print(f"📊 Confianza promedio: {promedio:.1f}%")
                    else:
                        print("⚠️ No hay registros")
                else:
                    print("⚠️ No existe la base de datos")

                input("\n[Presiona ENTER para continuar...]")

            elif opcion == '4':
                # CORREGIR REGISTRO
                print("\n" + "-" * 70)
                print("   OPCIÓN 4: CORREGIR REGISTRO")
                print("-" * 70 + "\n")

                if os.path.exists(ARCHIVO_EXCEL):
                    df = pd.read_excel(ARCHIVO_EXCEL)
                    if len(df) > 0:
                        print("Últimos 10 registros:")
                        display(df.tail(10))

                        print("\n📝 ¿Qué deseas hacer?\n")
                        print("  1. Corregir una placa específica")
                        print("  2. Eliminar un registro")
                        print("  3. Cancelar")

                        sub_opcion = input("\n➤ Opción (1-3): ").strip()

                        if sub_opcion == '1':
                            placa_buscar = input("\n🔍 Ingresa la placa INCORRECTA a buscar: ").strip().upper()

                            indices = df[df['Placa'] == placa_buscar].index.tolist()

                            if indices:
                                print(f"\n✓ Se encontraron {len(indices)} registro(s) con la placa '{placa_buscar}'")
                                display(df[df['Placa'] == placa_buscar])

                                placa_correcta = input("\n✏️  Ingresa la placa CORRECTA: ").strip().upper()

                                if placa_correcta and 5 <= len(placa_correcta) <= 8:
                                    confirmar = input(f"\n¿Cambiar '{placa_buscar}' por '{placa_correcta}'? (s/n): ").lower()

                                    if confirmar == 's':
                                        df.loc[indices, 'Placa'] = placa_correcta
                                        df.to_excel(ARCHIVO_EXCEL, index=False)
                                        print(f"\n✅ ¡{len(indices)} registro(s) corregido(s)!")
                                    else:
                                        print("\n❌ Corrección cancelada")
                                else:
                                    print("\n❌ Placa inválida (debe tener 5-8 caracteres)")
                            else:
                                print(f"\n⚠️ No se encontró la placa '{placa_buscar}'")

                        elif sub_opcion == '2':
                            numero_fila = input("\n🗑️  Ingresa el número de fila a eliminar (según la tabla): ").strip()

                            try:
                                idx = int(numero_fila)
                                if 0 <= idx < len(df):
                                    print(f"\nRegistro a eliminar:")
                                    print(f"  Fecha: {df.iloc[idx]['Fecha']}")
                                    print(f"  Hora: {df.iloc[idx]['Hora']}")
                                    print(f"  Placa: {df.iloc[idx]['Placa']}")

                                    confirmar = input("\n¿Confirmar eliminación? (s/n): ").lower()

                                    if confirmar == 's':
                                        df = df.drop(idx).reset_index(drop=True)
                                        df.to_excel(ARCHIVO_EXCEL, index=False)
                                        print("\n✅ ¡Registro eliminado!")
                                    else:
                                        print("\n❌ Eliminación cancelada")
                                else:
                                    print("\n❌ Número de fila inválido")
                            except:
                                print("\n❌ Número inválido")

                        else:
                            print("\n❌ Operación cancelada")
                    else:
                        print("⚠️ No hay registros para corregir")
                else:
                    print("⚠️ No existe la base de datos")

                input("\n[Presiona ENTER para continuar...]")

            elif opcion == '5':
                # DESCARGAR EXCEL
                print("\n" + "-" * 70)
                print("   OPCIÓN 5: DESCARGAR EXCEL")
                print("-" * 70 + "\n")

                if os.path.exists(ARCHIVO_EXCEL):
                    df = pd.read_excel(ARCHIVO_EXCEL)
                    print(f"📊 Total: {len(df)}\n")
                    if len(df) > 0:
                        display(df.tail(10))
                    print("\n📥 Descargando...")
                    files.download(ARCHIVO_EXCEL)
                    print("✅ ¡Completado!")
                else:
                    print("❌ No existe la base de datos")

                input("\n[Presiona ENTER para continuar...]")

            elif opcion == '6':
                # DESCARGAR IMÁGENES
                print("\n" + "-" * 70)
                print("   OPCIÓN 6: DESCARGAR IMÁGENES")
                print("-" * 70 + "\n")

                if os.path.exists(CARPETA_IMAGENES):
                    imagenes = os.listdir(CARPETA_IMAGENES)
                    if imagenes:
                        print(f"📸 Total: {len(imagenes)}\n")
                        opcion_desc = input("¿Descargar todas? (s/n): ").lower()

                        if opcion_desc == 's':
                            print("\n📥 Descargando...")
                            for img in imagenes:
                                files.download(os.path.join(CARPETA_IMAGENES, img))
                            print(f"✅ {len(imagenes)} imágenes descargadas!")
                    else:
                        print("⚠️ No hay imágenes")
                else:
                    print("❌ No existe la carpeta")

                input("\n[Presiona ENTER para continuar...]")

            elif opcion == '7':
                # SALIR
                print("\n" + "=" * 70)
                print("   👋 ¡GRACIAS POR USAR EL SISTEMA!")
                print("=" * 70)
                print(f"\n📊 Placas procesadas en esta sesión: {contador_sesion}")

                if os.path.exists(ARCHIVO_EXCEL):
                    df = pd.read_excel(ARCHIVO_EXCEL)
                    print(f"📈 Total en base de datos: {len(df)}")

                print("\n" + "=" * 70 + "\n")
                break

            else:
                print("\n⚠️ Opción inválida. Intenta de nuevo.")
                time.sleep(1)

        except KeyboardInterrupt:
            print("\n\n⚠️ Sistema interrumpido por el usuario")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            input("\n[Presiona ENTER para continuar...]")

# ============================================
# EJECUTAR SISTEMA AUTOMÁTICAMENTE
# ============================================

print("\n✅ Sistema cargado")
print("🚀 Iniciando menú principal...\n")
time.sleep(1)

sistema_principal()